In [3]:
import sys
sys.path.append('../')

from gears import PertData, GEARS

/deltadisk/miniconda3/envs/gears/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/deltadisk/miniconda3/envs/gears/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/deltadisk/miniconda3/envs/gears/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/deltadisk/miniconda3/envs/gears/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/deltadisk/miniconda3/envs/gears/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_

Load data. We use norman as an example.

In [ ]:
pert_data = PertData('./data')
pert_data.load(data_name = 'adamson')
pert_data.prepare_split(split = 'simulation', seed = 1)
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128)

Found local copy...
Found local copy...
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
['SRPR+ctrl' 'SLMO2+ctrl' 'TIMM23+ctrl' 'AMIGO3+ctrl' 'KCTD16+ctrl']
Local copy of pyg dataset is detected. Loading...
Done!
Local copy of split is detected. Loading...
Simulation split test composition:
combo_seen0:0
combo_seen1:0
combo_seen2:0
unseen_single:21
Done!
Creating dataloaders....
Done!


here1


Create a model object; if you use [wandb](https://wandb.ai), you can easily track model training and evaluation by setting `weight_bias_track` to true, and specify the `proj_name` and `exp_name` that you like.

In [8]:
gears_model = GEARS(pert_data, device = 'cuda:0', 
                        weight_bias_track = False, 
                        proj_name = 'pertnet', 
                        exp_name = 'pertnet')
gears_model.model_initialize(hidden_size = 64)

Found local copy...
Extracting tar file...
Done!


You can find available tunable parameters in model_initialize via

In [9]:
gears_model.tunable_parameters()

{'hidden_size': 'hidden dimension, default 64',
 'num_go_gnn_layers': 'number of GNN layers for GO graph, default 1',
 'num_gene_gnn_layers': 'number of GNN layers for co-expression gene graph, default 1',
 'decoder_hidden_size': 'hidden dimension for gene-specific decoder, default 16',
 'num_similar_genes_go_graph': 'number of maximum similar K genes in the GO graph, default 20',
 'num_similar_genes_co_express_graph': 'number of maximum similar K genes in the co expression graph, default 20',
 'coexpress_threshold': 'pearson correlation threshold when constructing coexpression graph, default 0.4',
 'uncertainty': 'whether or not to turn on uncertainty mode, default False',
 'uncertainty_reg': 'regularization term to balance uncertainty loss and prediction loss, default 1',
 'direction_lambda': 'regularization term to balance direction loss and prediction loss, default 1'}

Train your model:

Note: For the sake of demo, we set epoch size to 1. To get full model, set `epochs = 20`.

In [10]:
gears_model.train(epochs = 1, lr = 1e-3)

Start Training...


Epoch 1 Step 1 Train Loss: 0.5839
Epoch 1 Step 51 Train Loss: 0.5628
Epoch 1 Step 101 Train Loss: 0.5654
Epoch 1 Step 151 Train Loss: 0.6032
Epoch 1 Step 201 Train Loss: 0.5925
Epoch 1 Step 251 Train Loss: 0.5597
Epoch 1 Step 301 Train Loss: 0.5149
Epoch 1 Step 351 Train Loss: 0.6294
Epoch 1 Step 401 Train Loss: 0.6264
Epoch 1 Step 451 Train Loss: 0.5502
Epoch 1 Step 501 Train Loss: 0.4592
Epoch 1 Step 551 Train Loss: 0.5616
Epoch 1 Step 601 Train Loss: 0.6219
Epoch 1 Step 651 Train Loss: 0.5595
Epoch 1 Step 701 Train Loss: 0.5194
Epoch 1 Step 751 Train Loss: 0.4248
Epoch 1 Step 801 Train Loss: 0.5988
Epoch 1 Step 851 Train Loss: 0.5876
Epoch 1 Step 901 Train Loss: 0.5149
Epoch 1 Step 951 Train Loss: 0.5061
Epoch 1 Step 1001 Train Loss: 0.5330
Epoch 1 Step 1051 Train Loss: 0.4753
Epoch 1 Step 1101 Train Loss: 0.5616
Epoch 1 Step 1151 Train Loss: 0.4645
Epoch 1 Step 1201 Train Loss: 0.5409
Epoch 1 Step 1251 Train Loss: 0.5283
Epoch 1 Step 1301 Train Loss: 0.4777
Epoch 1 Step 1351 Train 

Save and load pretrained models:

In [11]:
gears_model.save_model('test_model')
gears_model.load_pretrained('test_model')

Make prediction for new perturbation:

In [12]:
gears_model.predict([['FEV'], ['FEV', 'AHR']])

{'FEV': array([ 5.9273574e-03, -8.6934875e-40,  5.7736444e-03, ...,
         1.3908881e-03,  2.1409787e-01,  2.8328854e-01], dtype=float32),
 'FEV_AHR': array([ 5.9247040e-03, -8.6938799e-40,  1.6300210e-03, ...,
        -1.7043231e-04,  2.0552994e-01,  3.1404313e-01], dtype=float32)}

Gene list can be found here:

In [13]:
gears_model.gene_list[:5]

['AP006222.2', 'RP11-54O7.16', 'RP11-54O7.1', 'RP11-54O7.3', 'SAMD11']